In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05190277099609375
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.394345663868785
200000 8.31009564217287
300000 9.826322274010813
400000 10.915097896772732
500000 11.667878548903449
600000 11.848706688404137
700000 10.956692087755918
800000 11.63381428630413
900000 14.009699122213457
1000000 16.988454757970114
1100000 16.47253680524848
1200000 9.677124083856405
1300000 21.769887832690795
1400000 22.437248519432895
1500000 22.185781169639956
1600000 22.28039559280014
1700000 22.943990378551867
1800000 22.17398239246889
1900000 22.26276430255719
2000000 21.87663459329748
2100000 22.537114452678267
2200000 21.86560425777282
2300000 5.715995608359155
2400000 25.246866228505848
2500000 26.983869535983057
2600000 27.40682286041459
2700000 26.159453556903742
2800000 27.242393920232868
2900000 26.90349084851953
3000000 25.4667

29400000 21.558572313664186
29500000 7.755928037793785
29600000 21.452166570547806
29700000 22.20083548131685
29800000 20.79069776944099
29900000 21.41587629467831
30000000 21.095460983417894
30100000 20.31628000981586
30200000 20.316436859566686
30300000 19.10028497030891
30400000 5.799272042944133
30500000 19.899011621596426
30600000 19.155339491499415
30700000 20.451648565036198
30800000 20.031283322795254
30900000 21.331486117582667
31000000 20.45261092526438
31100000 21.366818559638936
31200000 21.550146984976536
31300000 9.306188903980583
31400000 21.858092656318235
31500000 20.441434026201744
31600000 21.94845420982502
31700000 17.71940216430407
31800000 10.698586325349135
31900000 21.17061932228125
32000000 20.15725357106625
32100000 21.235181958272364
32200000 19.24666288840744
32300000 19.508543942022488
32400000 20.164893196806865
32500000 18.686636573557326
32600000 12.197961827774087
32700000 13.797739526266954
32800000 19.112227811168957
32900000 19.151722809934743
330000

59200000 17.77335352330461
59300000 15.840938537564814
59400000 17.092405671809324
59500000 16.297717811346313
59600000 7.43755164472725
59700000 15.61069826856977
59800000 16.21360312716837
59900000 16.999509622631848
60000000 16.347026438435417
60100000 17.310242868761033
60200000 17.508634856634156
60300000 7.600082601128138
60400000 19.237239328977935
60500000 18.006281980376052
60600000 18.709440943502106
60700000 18.414962470483612
60800000 18.505259491271907
60900000 18.898037295834516
61000000 18.272460747314035
61100000 19.125646039588414
61200000 8.150584573057198
61300000 16.20095028082017
61400000 17.469378232288737
61500000 16.175207847292597
61600000 16.846719325123757
61700000 16.148349494240467
61800000 14.474561404635097
61900000 8.529960598051519
62000000 15.758251655339466
62100000 16.492514850616963
62200000 15.916608001570413
62300000 17.035330996347316
62400000 16.803418721117637
62500000 9.349314971016932
62600000 16.489022768314644
62700000 17.58174435928475
628

88900000 10.38561912989527
89000000 14.647413202082049
89100000 14.639991199561203
89200000 14.604266184806407
89300000 6.619638247616914
89400000 15.306634293960656
89500000 15.210025953079233
89600000 15.619600664958467
89700000 15.786383034953058
89800000 16.445583548950225
89900000 16.138567100315722
90000000 16.574248088459832
90100000 16.002579610916207
90200000 16.291300347096914
90300000 16.292860721212787
90400000 15.609757633536217
90500000 14.943224402897341
90600000 11.905698716445347
90700000 8.437173757522016
90800000 14.156520162817477
90900000 14.212646800186882
91000000 15.088901123493633
91100000 4.2570071082135374
91200000 15.652029802254113
91300000 14.770454283813715
91400000 14.145615737475856
91500000 7.978265219617529
91600000 12.84088766501793
91700000 14.443842552850867
91800000 15.2312663926227
91900000 15.765353626322637
92000000 16.44193050406014
92100000 15.53214691053317
92200000 16.240745332720287
92300000 15.587443595825716
92400000 15.78564854945105
92

117900000 9.1117857007239
118000000 10.649986981912285
118100000 5.949773871734107
118200000 12.509996270615042
118300000 5.520890879670283
118400000 13.318586738533188
118500000 12.609136546426743
118600000 13.235807777076424
118700000 13.014733403714425
118800000 13.247162337552302
118900000 13.037706146961371
119000000 13.711063838931652
119100000 14.534060891424069
119200000 14.20455184006822
119300000 14.379184471094037
119400000 13.74388347621736
119500000 12.71298800482607
119600000 12.761965595143964
119700000 12.50353397488579
119800000 12.767503759259512
119900000 12.181079487535232
120000000 12.509091100617402
120100000 5.853850664246499
120200000 11.58338289511952
120300000 5.1864297232064045
120400000 11.053006340576678
120500000 7.157277456228099
120600000 10.919362703001323
120700000 12.541032141144226
120800000 12.795134943687287
120900000 12.118317370172543
121000000 12.75573787164865
121100000 12.34561923405355
121200000 12.881883667204853
121300000 13.831045274988682

...

175600000 8.834807500424098
175700000 8.643309579984201
175800000 8.263009301884026
175900000 8.715060426121582
176000000 8.305183586772028
176100000 8.484816436183326
176200000 8.328548081589789
176300000 3.6687729225082397
176400000 8.455059302237693
176500000 3.7860572265335324
176600000 7.324319309908563
176700000 5.411329439809147
176800000 6.014552873081952
176900000 8.568370350865669
177000000 8.192804783406649
177100000 8.657355514267833
177200000 8.264938085142319
177300000 8.714666080400967
177400000 8.74709287351488
177500000 9.196309341762541
177600000 9.493328161504891
177700000 9.388286340095231
177800000 9.147832622206227
177900000 8.848675717519873
178000000 8.59141927972278
178100000 8.477889016155741
178200000 8.62868787171038
178300000 8.23302847433571
178400000 8.294782062565284
178500000 4.991320299791899
178600000 7.271841412261098
178700000 7.060039269464132
178800000 4.057972137477931
178900000 7.985738125475824
179000000 3.6909173336928194
179100000 8.209319402

204900000 6.923387611139165
205000000 6.80978356324321
205100000 6.52604404170381
205200000 6.406565918395586
205300000 2.7611287802740834
205400000 6.472806371662505
205500000 6.298102898784463
205600000 6.394935556528797
205700000 4.266153386042926
205800000 4.588282528795421
205900000 6.399949852505296
206000000 6.471882789258563
206100000 6.462401608580053
206200000 2.461461599293093
206300000 6.5175667034534746
206400000 6.525369038646417
206500000 6.836280246553839
206600000 7.026288971004847
206700000 7.272615474071668
206800000 7.163680505275278
206900000 7.035590279545648
207000000 7.4069173781363675
207100000 7.201947403538247
207200000 6.916119986736023
207300000 6.890353897011849
207400000 6.380261265447896
207500000 4.003578110877958
207600000 5.389152737633231
207700000 6.392053236537694
207800000 6.361723809043109
207900000 6.659133266246315
208000000 1.851714670334667
208100000 6.7300593382076395
208200000 6.442078794327409
208300000 6.388033034141967
208400000 5.109903

234100000 5.182344041507215
234200000 4.79566516147699
234300000 1.9955708763186004
234400000 5.114869329202344
234500000 4.549734265919559
234600000 4.555017753811227
234700000 4.334641316780179
234800000 4.503621119161464
234900000 3.8190694659114217
235000000 2.08861201962016
235100000 4.545885924388319
235200000 4.189361401272595
235300000 4.5058996964443425
235400000 4.437529707968355
235500000 4.533617697044916
235600000 3.148015580587923
235700000 3.346834805733386
235800000 5.10436910199612
235900000 4.676436071850965
236000000 4.872877153109488
236100000 4.983839278807964
236200000 4.554388803055992
236300000 4.963590659331064
236400000 4.534328723692076
236500000 2.8953573413868248
236600000 3.770699706583573
236700000 4.398690943695005
236800000 4.317725916865597
236900000 4.079824051031546
237000000 4.330097954520397
237100000 3.8947574309987423
237200000 1.7776735468804108
237300000 3.8745802610760265
237400000 4.00634742121911
237500000 4.282952008258051
237600000 4.17478

263000000 2.6190692041505046
263100000 2.700471396077858
263200000 2.499463954452087
263300000 1.0296729074310065
263400000 2.617530615218775
263500000 2.459940420509875
263600000 2.546547255809969
263700000 2.462024340507259
263800000 2.4108884289491077
263900000 2.3670054821689988
264000000 2.222604817321877
264100000 2.3499294789039493
264200000 0.7036087059038678
264300000 2.4205204450278117
264400000 2.277868181321651
264500000 2.358148496645573
264600000 2.473624261271371
264700000 2.4134835310969254
264800000 2.543738120724831
264900000 2.4072055568088393
265000000 2.56097053884241
265100000 0.871041236820648
265200000 2.4563181216026444
265300000 2.6251305415195523
265400000 2.3756341332891218
265500000 1.542982539751013
265600000 1.9416878702408076
265700000 2.361686090972672
265800000 2.448210205041591
265900000 2.2485963257566297
266000000 2.447870319085194
266100000 2.2352572483745714
266200000 2.263807406636864
266300000 2.1256543885330226
266400000 1.4493354491670893
2665

291500000 0.44277644668701294
291600000 0.4430869989113311
291700000 0.4352809866922994
291800000 0.44181704747822875
291900000 0.43272631274432605
292000000 0.4120364194182157
292100000 0.42307946231834426
292200000 0.36006949254167747
292300000 0.08425110848677157
292400000 0.3167997390975422
292500000 0.3049474238883058
292600000 0.2979315153053105
292700000 0.2841378210508757
292800000 0.28451126055178044
292900000 0.28483075636486216
293000000 0.2642894655852086
293100000 0.25954648835973937
293200000 0.250329409281373
293300000 0.24614509229282538
293400000 0.10574065476707287
293500000 0.20251343917186063
293600000 0.22350853792217043
293700000 0.21810473413918746
293800000 0.21000482203390203
293900000 0.20245078583452436
294000000 0.20983996098839242
294100000 0.19108683534488083
294200000 0.1810195287921859
294300000 0.1805185744413899
294400000 0.1650584551280141
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.10355560412752296
294600000 0.